<a href="https://colab.research.google.com/github/SuleymanCanTugcu/Python/blob/main/botdeneme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ön Gereksinimler**


In [ ]:
!pip install python-binance #botu binance üzerinde kullanacağımız için  ekledik
!pip install catboost #catboost aynı google tensorflow gibi yapay zeka yapıları içeren(decision tree- ml vs) bir yapay zeka motoru.
!pip install matplotlib --upgrade #Grafiksel çizimler için ihtiyacımız olcak --upgrade yeni versiyonunu verir
!pip install mpl_finance #Bazı Finansal fonksiyonlar lazım olacak

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 KB 738.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 611.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 975.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Gerekli Kütüphaneler(Proje ilerledikçe ekleme-çıkarma yapılabilir!)**

In [ ]:
import pandas as pd #Data analizi için gerekli olan kütüphane
import numpy as np #C++ destekli matematiksel işlem yapabilen bir kütüphane
import time #API'den aldığımız data'daki zamanları kontrol etmek için gerekli kütüphane
import dateparser #Gelen zamanları saniye milisaniye gibi bölüp kullanmak için gerekli kütüphane
import pytz
import json #API içeriğini çekeceğimiz yapı(büyük ihtimalle)

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread #Multithread yapabilmek için gerekli kütüphane

import matplotlib.pyplot as plt #Strateji testleri için gerekli bir kütüphane
from mpl_finance import candlestick_ochl #Mum grafikleri çizmek için gerekli kütüphane
import matplotlib.dates as mdates
from matplotlib.pyplot import figure #Grafikleri iç içe çizdirebilmek için gerekli bir kütüphane

from sklearn import preprocessing, model_selection, neighbors, svm
#Scikit-Learn, makine öğrenmesi modeli için gerekli algoritmaların bulunduğu büyük bir kütüphane buradan:
#Preprocessing: Raw data'yı clean dataya dönüştürmek için olan konsept, modeli çalıştırmadan önce missing values, noisy data ve diğer tutarsız verileri düzeltmek için kullanılır.
#Svm: Support vector machine, supervised(gözetimli) öğrenme için sınıflama(classification) ve regresyon problemlerinde kullanılan algoritma.
#Model_selection: Final bir makine öğrenmesi modelini diğer aday modeller arasından, eğitilmek üzere seçme işlemini yapan algoritma.
#Neighbors: K-Nearest Neighbors algoritması; veriler hakkında yakınlık tahmini, sınıflama yapmak için kullanılan algoritma.

from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler #Aykırı değerleri birbine yaklaştırarak standardize etmede kullanılan ölçekleme yöntemleri.
#Scaler'lardan hangilerini kullanacağımız belirsiz. Modelin tutarlılığını iyileştirmek için düzeltme, ayar(tuning) yapınca hangilerinin ihtiyaç olduğu ortaya çıkacak.

from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error #İstatistiksel hata analizini yapabilmek için gerekli kütüphaneler.

from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv #Catboost Yandex'in Yapay Zeka motorunun kütüphanesinde, sınıflama, regresyon ve data havuzu gibi yapılar için gerekli kütüphaneler.

from tqdm import tqdm as tqdm #Progress bar için gerekli bir kütüphane(Opsiyonel)





# Financial Indicators


In [ ]:
#Finansal indikatör fonksiyonları bu kısımda tanımlanacak!

In [ ]:
def moving_average(df, n): # n = periyot df = dataframe
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))#mean=dizideki elemanların ortalaması
  df = df.join(MA)
  return df

In [ ]:
def exponential_moving_average(df, n):
  EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name='EMA_' + str(n))
  df = df.join(EMA)
  return df

In [ ]:
def momentum(df, n):
  M = pd.Series(df['Close'].diff(n), name="Momentum_" + str(n))
  df = df.join(M)
  return df 

In [ ]:
def bollinger_bands(df, n): #bollinger bantlarını ekliyeceğiz.BURAYI ARAŞTIRMAK ÖNEMLİ
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean())
  MSD = pd.Series(df['Close'].rolling(n, min_periods=n).std())
  b1 = 4 * MSD / MA
  B1 = pd.Series(b1, name = "BollingerB_" + str(n))
  df = df.Join(B1)
  
  b2 = (df['Close'] - MA + 2* MSD) / (4 * MSD)
  B2 = pd.Series(b2, name = "Bollinger%b_" + str(n))
  df = df.Join(B2)

  return df

In [ ]:
def stochastic_oscillator(df, n):
  S0K = pd.Series((df['Close'] - df['Low']) / (df['High'] - df['Low']), name='SO%k')
  df = df.join(S0K)
  return df

In [ ]:
def relative_strength_index(df, n):
  i = 0
  UpI = [0]
  DoI = [0]

  while i+1 <= df.index[-1]:
    UpMove = df.loc[i+1, 'High'] - df.loc[i, 'High']
    DoMove = df.loc[i+1, 'Low'] - df.loc[i, 'Low']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i + 1

  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
  df = df.join(RSI)
  return df

In [ ]:
def common_channel_index(df, n): #CCI
  PP = (df['High'] + df['Low'] + df['Close']) / 3 
  CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / PP.rolling(n, min_periods=n).std(), name='CCI_' + str(n))
  df = df.joint(CCI)
  return df

In [ ]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

In [ ]:
""" Örnek Deneme
df = pd.DataFrame(data = [1,2,3,4,5,6,7,8,9,100,233,123321,12121], columns=['Close'])
df = moving_average(df, 3)
df = moving_average(df, 6)
df """

" Örnek Deneme\ndf = pd.DataFrame(data = [1,2,3,4,5,6,7,8,9,100,233,123321,12121], columns=['Close'])\ndf = moving_average(df, 3)\ndf = moving_average(df, 6)\ndf "

# Keys Sections


https://testnet.binance.vision/

In [ ]:
!pip install python-binance #Binance API'ye erişebilmek için gerekli kütüphaneyi yüklüyoruz

binance_api= 'FlKCZZvYzkkOO7a7Vu390wSd7huqJvdfBxEmNRjMS5MKJ1gJM9RY5djF6qaXf9mD'
binance_secret= 'LaUi2QW1LIeavdLe4XK8PPVgfKOJ7kFeDoRrBvkp1V9fKP1TwrjGZHDIuRmlWwyN'

import os

from binance.client import Client

api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

client = Client(api_key, api_secret)
client.API_URL = 'https://testnet.binance.vision/api'

print(client.get_asset_balance(asset='USDT'))
#print(api_key)
#print(api_secret)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.9 MB/s eta 0:00:00


BinanceAPIException: ignored